In [ ]:
# Importing libraries
from ipywidgets import *
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams.update(mpl.rcParamsDefault)
import numpy as np
from scipy.ndimage import gaussian_filter1d 
%matplotlib notebook
import warnings
warnings.filterwarnings('ignore')

A simple model for a proton beam's depth-dose curve is<br>

$$D(x) = \frac{1}{\sqrt{d_o - x}}\ast \frac{1}{\sqrt{2\pi}\sigma}\text{exp}\left[\left(\frac{x}{\sigma}\right)^2\right],$$ <br>
where $D(x)$ is the dose at depth $x$, $d_0$ is the maximum depth, $\ast$ denotes a convolution, and the width of the Gaussian function $\sigma$ models the the range of energies in the proton beam.

For more details about this model and more accurate models see
Bortfeld, T. (1997), An analytical approximation of the Bragg curve for therapeutic proton beams. Med. Phys., 24: 2024-2033.

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(4,3))
ax.set_xlabel('Depth (cm)')
ax.set_ylabel('Relative Dose')
ax.set_xlim([0,5])
ax.set_ylim([0,1])
plt.tight_layout()

def D(x, d0):
    y = np.where(x < d0 - 1e-2, (d0 - x)**(-0.5), 0)
    return 0.1*gaussian_filter1d(y, sigma=5e-3*N)

N = 2000
x = np.linspace(0, 5, N)
linef, = ax.plot(x, D(x, 2.5), '-k')

def update(d0 = 2.5):
    linef.set_ydata(D(x, d0))

interact(update, d0=(1.0, 4.0, 0.01));

---
A single Bragg peak is not a very useful dose distribution for treating patients. We can create a "spread-out Bragg peak" by adding the weighted contributions from single Bragg peaks. 

TODO: Modify the weights and depths of the beamlets to create a spread-out Bragg peak between 2 and 3 cms. 

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(4,3))
ax.set_xlabel('Depth (cm)')
ax.set_ylabel('Relative Dose')
ax.set_xlim([0,5])
ax.set_ylim([0,1.1])
plt.tight_layout()

N = 2000
x = np.linspace(0, 5, N)
total_dose = np.zeros_like(x)

# Set weights and depths for proton beamlets
weights = [1, 1]
d0_list = [2.0, 2.5]

# Plot beamlets
for i, d0 in enumerate(d0_list):
    dose = weights[i]*D(x, d0)
    ax.plot(x, dose, '-', lw=0.5)
    total_dose += dose
    
# Plot total
ax.plot(x, total_dose/np.max(total_dose), '-k')